### Model Fine-Tuning
BERT base uncased

In [2]:
!pip install transformers datasets peft evaluate accelerate bitsandbytes pandas -q
!pip install optuna scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.1 MB/s eta 0:00:00


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Basic fine-tuning script

In [11]:
import os
import argparse
from typing import Dict, Any, List

import torch
from datasets import load_dataset, Dataset, ClassLabel, Features, Value
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np
import pandas as pd
import json

# -------------------------
# Configuration
# -------------------------
class SimpleArgs:
    csv = "/content/drive/MyDrive/ANLP Assignment 1/mt_bench_training.csv"
    output_dir = "./model_adapter"
    model_name = "bert-base-uncased"  # <-- switched to encoder-only
    batch_size = 8
    epochs = 6
    lr = 2e-5
    seed = 42
    max_input_length = 512
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.05
    save_total_limit = 2
    eval_steps = 10
    logging_steps = 10
    seed_data_split = 42
    test_size = 0.1
    validation_size = 0.1

args = SimpleArgs()

# -------------------------
# Utilities
# -------------------------
def build_input_texts_from_columns(examples: Dict[str, List], tokenizer) -> List[str]:
    text_inputs = []
    sep_token = tokenizer.sep_token if tokenizer.sep_token is not None else " "

    for i in range(len(examples["turn"])):
        turn = int(examples["turn"][i])
        q1 = str(examples.get("turn_1_query", [""])[i]).strip()

        if turn == 1:
            text = f"Query: {q1}"
        elif turn == 2:
            ans = str(examples.get("turn_1_answer", [""])[i]).strip()
            q2 = str(examples.get("turn_2_query", [""])[i]).strip()
            text = f"Query: {q1}{sep_token}Answer: {ans}{sep_token}Follow-up Query: {q2}"
        else:
            text = f"Query: {q1}"
        text_inputs.append(text)
    return text_inputs


def preprocess_function(examples, tokenizer, args):
    text_inputs = build_input_texts_from_columns(examples, tokenizer)

    model_inputs = tokenizer(
        text_inputs,
        max_length=args.max_input_length,
        truncation=True,
        padding=False,
    )
    model_inputs["labels"] = examples["label"]
    return model_inputs


# -------------------------
# Compute Metrics
# -------------------------
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        logits = preds[0]
    else:
        logits = preds

    pred_ids = np.argmax(logits, axis=1)
    acc = accuracy_metric.compute(predictions=pred_ids, references=labels)
    return {"accuracy": acc["accuracy"]}

# -------------------------
# Main Logic
# -------------------------
def main():
    torch.manual_seed(args.seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"--- Using device: {device.upper()} ---")

    if not os.path.exists(args.csv):
        raise FileNotFoundError(f"CSV file not found: {args.csv}")

    df = pd.read_csv(args.csv)

    unique_winners = sorted(df["winner"].unique().tolist())
    label2id = {label: i for i, label in enumerate(unique_winners)}
    id2label = {i: label for i, label in enumerate(unique_winners)}
    num_labels = len(unique_winners)
    print(f"Found {num_labels} unique labels: {unique_winners}")
    print(f"Label mapping: {label2id}")

    # Save the label mappings alongside the model adapter for inference
    mappings_dir = "/content/drive/MyDrive/ANLP Assignment 1"
    mappings_path = os.path.join(mappings_dir, "label_mappings.json")
    os.makedirs(mappings_dir, exist_ok=True) # Create the directory if it doesn't exist
    with open(mappings_path, "w") as f:
        json.dump({"id2label": id2label, "label2id": label2id}, f)
    print(f"Label mappings saved to {mappings_path}")
    # --------------------
    df['label'] = df['winner'].map(label2id)

    features = Features({
        'question_id': Value('int64'),
        'turn': Value('int64'),
        'turn_1_query': Value('string'),
        'turn_1_answer': Value('string'),
        'turn_2_query': Value('string'),
        'winner': Value('string'),
        'label': ClassLabel(names=unique_winners)
    })

    raw_all = Dataset.from_pandas(df, features=features)

    train_val_split = raw_all.train_test_split(
        test_size=args.test_size,
        seed=args.seed_data_split,
        stratify_by_column="label"
    )
    test_ds = train_val_split["test"]
    train_val_ds = train_val_split["train"]

    train_split = train_val_ds.train_test_split(
        test_size=args.validation_size,
        seed=args.seed_data_split,
        stratify_by_column="label"
    )
    train_ds = train_split["train"]
    val_ds = train_split["test"]

    print(f"Dataset splits created: train={len(train_ds)}, validation={len(val_ds)}, test={len(test_ds)}")

    tokenizer = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )

    # For BERT, we usually target query/key/value/projection layers for LoRA
    target_modules = ["query", "key", "value", "dense"]

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        lora_dropout=args.lora_dropout,
        target_modules=target_modules,
    )

    model = get_peft_model(model, peft_config)
    print("Wrapped model with LoRA. Trainable parameters:")
    model.print_trainable_parameters()

    tokenized_train = train_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)
    tokenized_val = val_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)
    tokenized_test = test_ds.map(lambda examples: preprocess_function(examples, tokenizer, args), batched=True)

    columns_to_remove = ['question_id', 'turn', 'turn_1_query', 'turn_1_answer', 'turn_2_query', 'winner', 'label']
    tokenized_train = tokenized_train.remove_columns([col for col in columns_to_remove if col in tokenized_train.column_names])
    tokenized_val = tokenized_val.remove_columns([col for col in columns_to_remove if col in tokenized_val.column_names])
    tokenized_test = tokenized_test.remove_columns([col for col in columns_to_remove if col in tokenized_test.column_names])

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=args.output_dir,
        eval_strategy="steps",
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.epochs,
        learning_rate=args.lr,
        save_total_limit=args.save_total_limit,
        fp16=torch.cuda.is_available(),
        logging_steps=args.logging_steps,
        eval_steps=args.eval_steps,
        save_strategy="steps",
        save_steps=args.eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        seed=args.seed,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("--- Starting Training ---")
    trainer.train()
    print("--- Training Finished ---")

    print("\n--- Evaluating on the held-out Test Set ---")
    test_results = trainer.evaluate(eval_dataset=tokenized_test)
    print("Test Set Metrics:")
    print(test_results)

    print("\nSaving final PEFT adapter to:", args.output_dir)
    model.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)
    print("Done.")

# Run the main function
main()

--- Using device: CUDA ---
Found 6 unique labels: ['alpaca-13b', 'claude-v1', 'gpt-3.5-turbo', 'gpt-4', 'llama-13b', 'vicuna-13b-v1.2']
Label mapping: {'alpaca-13b': 0, 'claude-v1': 1, 'gpt-3.5-turbo': 2, 'gpt-4': 3, 'llama-13b': 4, 'vicuna-13b-v1.2': 5}
Label mappings saved to /content/drive/MyDrive/ANLP Assignment 1/label_mappings.json
Dataset splits created: train=3348, validation=373, test=414


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wrapped model with LoRA. Trainable parameters:
trainable params: 2,683,398 || all params: 112,170,252 || trainable%: 2.3923


Map:   0%|          | 0/3348 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2543665550.py:209: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Starting Training ---


Step,Training Loss,Validation Loss,Accuracy
10,No log,1.881034,0.203753
20,No log,1.849910,0.203753
30,No log,1.822829,0.203753
40,No log,1.797495,0.206434
50,1.880700,1.781420,0.209115


KeyboardInterrupt: 

#### Better fine-tuning strategy

In [21]:
import os
import json
from typing import Dict, Any, List

import torch
import torch.nn as nn
from datasets import Dataset, Features, Value, ClassLabel # <-- Imports are correct
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np
import pandas as pd
import optuna
from sklearn.utils.class_weight import compute_class_weight

# (Configuration and utility functions remain the same)
class SimpleArgs:
    csv = "/content/drive/MyDrive/ANLP Assignment 1/mt_bench_training.csv"
    output_dir = "./hpo_results"
    model_name = "bert-base-uncased"
    batch_size = 16
    epochs = 8
    lr = 2e-5
    seed = 42
    max_input_length = 512
    lora_r = 16
    lora_alpha = 32
    lora_dropout = 0.1
    save_total_limit = 2
    eval_steps = 50
    logging_steps = 50
    seed_data_split = 42
    test_size = 0.1
    validation_size = 0.1
    n_hpo_trials = 20

args = SimpleArgs()

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def build_input_texts_from_columns(examples: Dict[str, List], tokenizer) -> List[str]:
    text_inputs = []
    sep_token = tokenizer.sep_token if tokenizer.sep_token is not None else " "
    for i in range(len(examples["turn"])):
        turn, q1 = int(examples["turn"][i]), str(examples.get("turn_1_query", [""])[i]).strip()
        if turn == 2:
            ans, q2 = str(examples.get("turn_1_answer", [""])[i]).strip(), str(examples.get("turn_2_query", [""])[i]).strip()
            text = f"Query: {q1}{sep_token}Answer: {ans}{sep_token}Follow-up Query: {q2}"
        else:
            text = f"Query: {q1}"
        text_inputs.append(text)
    return text_inputs

def preprocess_function(examples, tokenizer, args):
    text_inputs = build_input_texts_from_columns(examples, tokenizer)
    model_inputs = tokenizer(text_inputs, max_length=args.max_input_length, truncation=True, padding=False)
    model_inputs["labels"] = examples["label"]
    return model_inputs

accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    logits = preds[0] if isinstance(preds, tuple) else preds
    pred_ids = np.argmax(logits, axis=1)
    acc = accuracy_metric.compute(predictions=pred_ids, references=labels)
    return {"accuracy": acc["accuracy"]}


# -------------------------
# Main Logic
# -------------------------
def main():
    torch.manual_seed(args.seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"--- Using device: {device.upper()} ---")

    df = pd.read_csv(args.csv)
    # Fill NaN values in text columns to prevent errors
    text_cols = ['turn_1_query', 'turn_1_answer', 'turn_2_query']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna('')

    unique_winners = sorted(df["winner"].unique().tolist())
    label2id = {label: i for i, label in enumerate(unique_winners)}
    id2label = {i: label for i, label in enumerate(unique_winners)}
    num_labels = len(unique_winners)
    df['label'] = df['winner'].map(label2id)

    # --- THIS IS THE FIX ---
    # Define the features of the dataset, marking 'label' as a ClassLabel
    # This is necessary for stratified splitting.
    features = Features({
        'turn': Value('int64'),
        'turn_1_query': Value('string'),
        'turn_1_answer': Value('string'),
        'turn_2_query': Value('string'),
        'winner': Value('string'),
        'label': ClassLabel(names=unique_winners)
    })
    # Remove any columns from the DataFrame that are not in the Features
    df_columns = [col for col in features.keys() if col in df.columns]
    df = df[df_columns]

    # Create the dataset using the defined features
    raw_all = Dataset.from_pandas(df, features=features)
    # --- END FIX ---

    # Data Splitting
    train_val_split = raw_all.train_test_split(test_size=args.test_size, seed=args.seed_data_split, stratify_by_column="label")
    test_ds = train_val_split["test"]
    train_val_ds = train_val_split["train"]
    train_split = train_val_ds.train_test_split(test_size=args.validation_size, seed=args.seed_data_split, stratify_by_column="label")
    train_ds = train_split["train"]
    val_ds = train_split["test"]

    tokenizer = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)

    # Calculate Class Weights
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_ds['label']),
        y=np.array(train_ds['label'])
    )
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
    print(f"Calculated class weights for weighted loss: {class_weights}")

    def model_init(trial):
        return AutoModelForSequenceClassification.from_pretrained(
            args.model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
        )

    def optuna_hp_space(trial: optuna.Trial) -> Dict[str, Any]:
        return {
            "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
            "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16]),
            "lora_r": trial.suggest_categorical("lora_r", [8, 16, 32]),
            "lora_alpha": trial.suggest_categorical("lora_alpha", [16, 32, 64]),
        }

    tokenized_train = train_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)
    tokenized_val = val_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)
    tokenized_test = test_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=os.path.join(args.output_dir, "base_run"),
        eval_strategy="steps",
        per_device_eval_batch_size=args.batch_size,
        save_total_limit=args.save_total_limit,
        fp16=torch.cuda.is_available(),
        logging_steps=args.logging_steps,
        eval_steps=args.eval_steps,
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        seed=args.seed,
    )

    trainer = WeightedLossTrainer(
        model=None, args=training_args,
        train_dataset=tokenized_train, eval_dataset=tokenized_val,
        tokenizer=tokenizer, data_collator=data_collator,
        compute_metrics=compute_metrics, model_init=model_init,
        class_weights=class_weights_tensor,
    )

    print("--- Starting Hyperparameter Search ---")
    best_run = trainer.hyperparameter_search(
        direction="maximize", backend="optuna",
        hp_space=optuna_hp_space, n_trials=args.n_hpo_trials,
    )
    print("--- Hyperparameter Search Finished ---")
    print("Best Run Found:", best_run)

    print("\n--- Training Final Model with Best Hyperparameters ---")
    best_params = best_run.hyperparameters

    final_model = model_init(None)
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=best_params.get("lora_r", args.lora_r),
        lora_alpha=best_params.get("lora_alpha", args.lora_alpha),
        lora_dropout=args.lora_dropout,
        target_modules=["query", "value", "key", "dense"],
    )
    final_model = get_peft_model(final_model, peft_config)

    final_training_args = TrainingArguments(
        output_dir=os.path.join(args.output_dir, "final_model"),
        learning_rate=best_params.get("learning_rate"),
        num_train_epochs=best_params.get("num_train_epochs"),
        per_device_train_batch_size=best_params.get("per_device_train_batch_size"),
        eval_strategy="epoch", save_strategy="epoch",
        per_device_eval_batch_size=args.batch_size,
        save_total_limit=args.save_total_limit,
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        seed=args.seed,
    )

    final_trainer = WeightedLossTrainer(
        model=final_model, args=final_training_args,
        train_dataset=tokenized_train, eval_dataset=tokenized_val,
        tokenizer=tokenizer, data_collator=data_collator,
        compute_metrics=compute_metrics, class_weights=class_weights_tensor,
    )

    final_trainer.train()
    print("--- Final Training Finished ---")

    print("\n--- Evaluating Final Model on Test Set ---")
    test_results = final_trainer.evaluate(eval_dataset=tokenized_test)
    print("Final Test Set Metrics:", test_results)

    final_model_dir = os.path.join(args.output_dir, "final_model_adapter")
    final_trainer.save_model(final_model_dir)
    tokenizer.save_pretrained(final_model_dir)
    with open(os.path.join(final_model_dir, "label_mappings.json"), "w") as f:
        json.dump({"id2label": {str(k): v for k, v in id2label.items()}, "label2id": label2id}, f)
    print(f"Final optimized model and artifacts saved to {final_model_dir}")

if __name__ == "__main__":
    main()

--- Using device: CUDA ---
Calculated class weights for weighted loss: [1.69604863 0.9        0.59298618 0.79714286 2.66985646 1.01639344]


Map:   0%|          | 0/3348 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1363416978.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-09-16 10:13:30,701] A new study created in memory with name: no-name-7b55618a-06c5-4dee-a2c8-7bec23cb5443
Trying to set lora_r in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set lora_alpha in the hyperparameter search but there i

...

--- Hyperparameter Search Finished ---
Best Run Found: BestRun(run_id='11', objective=0.4772117962466488, hyperparameters={'learning_rate': 2.3969166892094607e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 16, 'lora_r': 32, 'lora_alpha': 64}, run_summary=None)

--- Training Final Model with Best Hyperparameters ---


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1363416978.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.776529,0.179625
2,No log,1.740171,0.184987
3,1.770600,1.707468,0.190349
4,1.770600,1.696927,0.198391


--- Final Training Finished ---

--- Evaluating Final Model on Test Set ---


Final Test Set Metrics: {'eval_loss': 1.7319389581680298, 'eval_accuracy': 0.21497584541062803, 'eval_runtime': 4.4217, 'eval_samples_per_second': 93.63, 'eval_steps_per_second': 5.88, 'epoch': 4.0}
Final optimized model and artifacts saved to ./hpo_results/final_model_adapter


#### Third attempt


In [5]:
import os
import json
from typing import Dict, Any, List

import torch
import torch.nn as nn
from datasets import Dataset, Features, Value, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np
import pandas as pd
import optuna
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler

# -------------------------
# Config
# -------------------------
class SimpleArgs:
    csv = "/content/drive/MyDrive/ANLP Assignment 1/mt_bench_training.csv"
    output_dir = "./hpo_results"
    model_name = "bert-base-uncased"
    batch_size = 16
    epochs = 10
    lr = 3e-5
    seed = 42
    max_input_length = 256
    lora_r = 32
    lora_alpha = 64
    lora_dropout = 0.2
    save_total_limit = 2
    eval_steps = 100
    logging_steps = 50
    seed_data_split = 42
    test_size = 0.1
    validation_size = 0.1
    n_hpo_trials = 15

args = SimpleArgs()

# -------------------------
# Trainer with weighted loss
# -------------------------
class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# -------------------------
# Preprocessing
# -------------------------
def build_input_texts_from_columns(examples: Dict[str, List], tokenizer) -> List[str]:
    text_inputs = []
    sep_token = tokenizer.sep_token if tokenizer.sep_token is not None else " "
    for i in range(len(examples["turn"])):
        turn, q1 = int(examples["turn"][i]), str(examples.get("turn_1_query", [""])[i]).strip()
        if turn == 2:
            ans, q2 = str(examples.get("turn_1_answer", [""])[i]).strip(), str(examples.get("turn_2_query", [""])[i]).strip()
            text = f"[Q1] {q1} {sep_token} [A1] {ans} {sep_token} [Q2] {q2}"
        else:
            text = f"[Q1] {q1}"
        text_inputs.append(text)
    return text_inputs

def preprocess_function(examples, tokenizer, args):
    text_inputs = build_input_texts_from_columns(examples, tokenizer)
    model_inputs = tokenizer(text_inputs, max_length=args.max_input_length, truncation=True, padding=False)
    model_inputs["labels"] = examples["label"]
    return model_inputs

# -------------------------
# Metrics
# -------------------------
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    logits = preds[0] if isinstance(preds, tuple) else preds
    pred_ids = np.argmax(logits, axis=1)
    acc = accuracy_metric.compute(predictions=pred_ids, references=labels)
    f1 = f1_metric.compute(predictions=pred_ids, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "macro_f1": f1["f1"]}

# -------------------------
# Main
# -------------------------
def main():
    torch.manual_seed(args.seed)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"--- Using device: {device.upper()} ---")

    df = pd.read_csv(args.csv)
    df.fillna("", inplace=True)

    unique_winners = sorted(df["winner"].unique().tolist())
    label2id = {label: i for i, label in enumerate(unique_winners)}
    id2label = {i: label for i, label in enumerate(unique_winners)}
    num_labels = len(unique_winners)
    df['label'] = df['winner'].map(label2id)

    features = Features({
        'turn': Value('int64'),
        'turn_1_query': Value('string'),
        'turn_1_answer': Value('string'),
        'turn_2_query': Value('string'),
        'winner': Value('string'),
        'label': ClassLabel(names=unique_winners)
    })
    df = df[[col for col in features.keys() if col in df.columns]]
    raw_all = Dataset.from_pandas(df, features=features)

    # Splitting
    train_val_split = raw_all.train_test_split(test_size=args.test_size, seed=args.seed_data_split, stratify_by_column="label")
    test_ds = train_val_split["test"]
    train_val_ds = train_val_split["train"]
    train_split = train_val_ds.train_test_split(test_size=args.validation_size, seed=args.seed_data_split, stratify_by_column="label")
    train_ds, val_ds = train_split["train"], train_split["test"]

    tokenizer = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)

    # Class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(train_ds['label']), y=np.array(train_ds['label']))
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

    # Tokenization
    tokenized_train = train_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)
    tokenized_val = val_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)
    tokenized_test = test_ds.map(lambda ex: preprocess_function(ex, tokenizer, args), batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Weighted sampler (helps with imbalance!)
    class_sample_counts = np.bincount(np.array(train_ds["label"]))
    weights = 1.0 / class_sample_counts
    sample_weights = [weights[label] for label in train_ds["label"]]
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    # Model
    base_model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
    )
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        lora_dropout=args.lora_dropout,
        target_modules=["query", "key", "value", "dense", "output.dense", "intermediate.dense"]
    )
    model = get_peft_model(base_model, peft_config)

    training_args = TrainingArguments(
        output_dir=os.path.join(args.output_dir, "final_model"),
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=args.lr,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.epochs,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=args.logging_steps,
        save_total_limit=args.save_total_limit,
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        fp16=torch.cuda.is_available(),
        seed=args.seed,
    )

    trainer = WeightedLossTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor,
    )

    trainer.train()
    print("--- Training Finished ---")

    print("\n--- Evaluating on Test Set ---")
    test_results = trainer.evaluate(eval_dataset=tokenized_test)
    print("Final Test Set Metrics:", test_results)

    trainer.save_model(os.path.join(args.output_dir, "final_model"))
    tokenizer.save_pretrained(os.path.join(args.output_dir, "final_model"))
    with open(os.path.join(args.output_dir, "label_mappings.json"), "w") as f:
        json.dump({"id2label": {str(k): v for k, v in id2label.items()}, "label2id": label2id}, f)

if __name__ == "__main__":
    main()


--- Using device: CUDA ---


Map:   0%|          | 0/3348 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1314363363.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.800500,1.769526,0.190349,0.157152
2,1.762400,1.730669,0.238606,0.218082
3,1.717100,1.698591,0.238606,0.230447
4,1.686400,1.665323,0.262735,0.214197
5,1.665900,1.633288,0.252011,0.245033
6,1.656700,1.605794,0.284182,0.275553
7,1.588100,1.610000,0.302949,0.292661
8,1.597800,1.587612,0.292225,0.273432
9,1.600200,1.585137,0.265416,0.259610
10,1.548700,1.586426,0.300268,0.291104


--- Training Finished ---

--- Evaluating on Test Set ---


Final Test Set Metrics: {'eval_loss': 1.6440895795822144, 'eval_accuracy': 0.26570048309178745, 'eval_macro_f1': 0.25283914100351107, 'eval_runtime': 2.4529, 'eval_samples_per_second': 168.779, 'eval_steps_per_second': 10.6, 'epoch': 10.0}


#### Model Saving to google drive functionality

In [6]:


# Define the path in Google Drive where you want to save the model
# You can change 'my_finetuned_model' to a different folder name if you prefer
GOOGLE_DRIVE_SAVE_PATH = '/content/drive/MyDrive/ANLP Assignment 1/Model File'

# Create the directory in Google Drive if it doesn't exist
os.makedirs(GOOGLE_DRIVE_SAVE_PATH, exist_ok=True)

print(f"Google Drive mounted at /content/drive")
print(f"Model will be saved to {GOOGLE_DRIVE_SAVE_PATH}")

Google Drive mounted at /content/drive
Model will be saved to /content/drive/MyDrive/ANLP Assignment 1/Model File


In [8]:
# Define the directory where the model adapter is saved locally after training
LOCAL_MODEL_DIR = "./hpo_results/final_model"

# Define the destination path in Google Drive
GOOGLE_DRIVE_DEST_DIR = GOOGLE_DRIVE_SAVE_PATH

# Copy the entire model adapter directory to Google Drive
if os.path.exists(LOCAL_MODEL_DIR):
    # Remove the destination directory in Google Drive if it already exists to avoid errors during copy
    if os.path.exists(GOOGLE_DRIVE_DEST_DIR):
        print(f"Removing existing directory in Google Drive: {GOOGLE_DRIVE_DEST_DIR}")
        shutil.rmtree(GOOGLE_DRIVE_DEST_DIR)

    print(f"Copying model adapter from {LOCAL_MODEL_DIR} to {GOOGLE_DRIVE_DEST_DIR}")
    shutil.copytree(LOCAL_MODEL_DIR, GOOGLE_DRIVE_DEST_DIR)
    print("Model adapter successfully saved to Google Drive.")
else:
    print(f"Local model directory not found: {LOCAL_MODEL_DIR}. Please run the training code first.")

Removing existing directory in Google Drive: /content/drive/MyDrive/ANLP Assignment 1/Model File
Copying model adapter from ./hpo_results/final_model to /content/drive/MyDrive/ANLP Assignment 1/Model File
Model adapter successfully saved to Google Drive.


### Model Inference (loads the fine-tuned model previously saved in google drive)

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
import json
import os
from google.colab import drive

# Mount Google Drive if it's not already mounted
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define the path in Google Drive where the model is saved
GOOGLE_DRIVE_SAVED_MODEL_PATH = '/content/drive/MyDrive/ANLP Assignment 1/Model File' # Make sure this matches the save path

def predict(query: str, model_dir: str):
    """
    Loads a PEFT model and tokenizer from a directory and performs inference.

    Args:
        query (str): The input text (user query) to classify.
        model_dir (str): The directory containing the PEFT adapter and artifacts (in Google Drive).

    Returns:
        str: The predicted class label (the best model name).
    """
    # --- 1. Load All Artifacts from the Directory ---

    # Load the PEFT config to get the base model name
    config = PeftConfig.from_pretrained(model_dir)
    base_model_name = config.base_model_name_or_path

    # Load the label mappings
    mappings_path = os.path.join('/content/drive/MyDrive/ANLP Assignment 1/', "label_mappings.json")
    with open(mappings_path, "r") as f:
        label_mappings = json.load(f)
        # The keys in the JSON file are strings, convert them back to integers
        id2label = {int(k): v for k, v in label_mappings["id2label"].items()}
        label2id = label_mappings["label2id"]

    num_labels = len(id2label)

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_dir)

    # --- 2. Build the Model ---

    # Load the base model with the correct classification head
    base_model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )

    # Apply the LoRA adapter
    model = PeftModel.from_pretrained(base_model, model_dir)
    model.eval()

    # --- 3. Perform Inference ---
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    predicted_id = torch.argmax(logits, dim=-1).item()
    predicted_label = model.config.id2label[predicted_id]

    return predicted_label




In [25]:
if __name__ == '__main__':
    # Use the Google Drive path for inference
    ADAPTER_DIRECTORY = GOOGLE_DRIVE_SAVED_MODEL_PATH

    # Example query for inference
    test_query = "Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words."

    # Get the prediction
    best_model = predict(test_query, ADAPTER_DIRECTORY)

    print(f"Input Query:\n'{test_query}'")
    print("---")
    print(f"Predicted Best Model: {best_model}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input Query:
'Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words.'
---
Predicted Best Model: claude-v1
